# Introdução ao RAG (Retrieval-Augmented Generation)

## 1. RAG
RAG, ou Retrieval Augmented Generation, é uma técnica que combina os recursos de um large language model pré-treinado com uma fonte de dados externa. Essa abordagem combina o poder generativo de LLMs com a precisão de mecanismos especializados de busca de dados, resultando em um sistema que pode oferecer respostas diferenciadas.

## 2. Carregando documentos para RAG

In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# carregando um arquivo CSV
csv_loader = CSVLoader(file_path="sleep_cycle_productivity.csv")
documents = csv_loader.load()

print(documents)


[Document(metadata={'source': 'sleep_cycle_productivity.csv', 'row': 0}, page_content='Date: 2024-04-12\nPerson_ID: 1860\nAge: 32\nGender: Other\nSleep Start Time: 23.33\nSleep End Time: 4.61\nTotal Sleep Hours: 5.28\nSleep Quality: 3\nExercise (mins/day): 86\nCaffeine Intake (mg): 87\nScreen Time Before Bed (mins): 116\nWork Hours (hrs/day): 8.80892009394567\nProductivity Score: 8\nMood Score: 3\nStress Level: 6'), Document(metadata={'source': 'sleep_cycle_productivity.csv', 'row': 1}, page_content='Date: 2024-11-04\nPerson_ID: 1769\nAge: 41\nGender: Female\nSleep Start Time: 21.02\nSleep End Time: 2.43\nTotal Sleep Hours: 5.41\nSleep Quality: 5\nExercise (mins/day): 32\nCaffeine Intake (mg): 21\nScreen Time Before Bed (mins): 88\nWork Hours (hrs/day): 6.329833121584335\nProductivity Score: 10\nMood Score: 3\nStress Level: 7'), Document(metadata={'source': 'sleep_cycle_productivity.csv', 'row': 2}, page_content='Date: 2024-08-31\nPerson_ID: 2528\nAge: 20\nGender: Male\nSleep Start Tim

In [2]:
from langchain_community.document_loaders import PyPDFLoader

# carregando um arquivo PDF
pdf_loader = PyPDFLoader("contosFluminenses.pdf")
documents = pdf_loader.load()

print(documents)


[Document(metadata={'producer': 'Acrobat Distiller 8.0.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2008-09-10T15:14:40-03:00', 'author': 'Deise', 'moddate': '2008-09-10T15:14:40-03:00', 'title': 'Microsoft Word - CONTO, Contos Fluminenses, 1870', 'source': 'contosFluminenses.pdf', 'total_pages': 164, 'page': 0, 'page_label': '1'}, page_content='Contos Fluminenses \n  \n \n  \nTexto-fonte: \nObra Completa, Machado de Assis, vol. II, \nRio de Janeiro: Nova Aguilar, 1994. \n  \nPublicado originalmente pela Editora Garnier, Rio de Janeiro, em 1870. \n  \n  \n  \n  \nÍNDICE \n  \n  \nMISS DOLLAR \n  \nLUÍS SOARES \n  \nA MULHER DE PRETO \n  \nO SEGREDO DE AUGUSTA \n  \nCONFISSÕES DE UMA VIÚVA MOÇA \n  \nLINHA RETA E LINHA CURVA \n  \nFREI SIMÃO \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \nMISS DOLLAR \n  \n  \n  \n  \nÍNDICE \n  \n  \nCAPÍTULO PRIMEIRO \n  \nCAPÍTULO II \n  \nCAPÍTULO III \n  \nCAPÍTULO IV \n  \nCAPÍTULO V'), Document(metadata={'producer': 'Acrobat 

## 3. Processamento e armazenamento de texto

Após carregar os documentos, é necessário dividi-los em chunks e criar embeddings para armazená-los em um banco de dados vetorial.

In [3]:
from langchain_text_splitters import CharacterTextSplitter

text = """Machine learning é um campo fascinante. 
Envolve algoritmos e modelos que aprendem com dados. 
Esses modelos podem fazer previsões sem serem explicitamente programados."""

# Dividindo o texto em pedaços menores
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=50, chunk_overlap=10)
chunks = text_splitter.split_text(text)

print(chunks)


Created a chunk of size 53, which is longer than the specified 50


['Machine learning é um campo fascinante.', 'Envolve algoritmos e modelos que aprendem com dados.', 'Esses modelos podem fazer previsões sem serem explicitamente programados.']


## 4. Armazenando chunks como embeddings

Os chunks são transformados em vetores para armazená-los em um banco de dados vetorial como ChromaDB.

In [4]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_ollama import OllamaEmbeddings
 

def get_embedding_function():
    embeddings = OllamaEmbeddings(model="snowflake-arctic-embed2")
    return embeddings
 
# texto de exemplo
text = """Machine learning é um campo fascinante. 
Envolve algoritmos e modelos que aprendem com dados. 
Esses modelos podem fazer previsões sem serem explicitamente programados."""

# dividindo o texto em chunks
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=50, chunk_overlap=10)
chunks = text_splitter.split_text(text)

# cnvertendo chunks para objetos Document
documents = [Document(page_content=chunk) for chunk in chunks]

vector_store = Chroma.from_documents(documents=documents, embedding=get_embedding_function())

Created a chunk of size 53, which is longer than the specified 50


## 5. Recuperando informações

Após armazenar os chunks, podemos recuperá-los com base em similaridade semântica.

### 5.1 Criando um retriever

In [5]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 2})

### 5.2 Criando um modelo de prompt

In [6]:
import os
from dotenv import load_dotenv

# configuração do modelo: mude para a sua chave de API do Google
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')

In [7]:
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

prompt = ChatPromptTemplate.from_template(
    "Com base no contexto abaixo, responda à pergunta:\n\nContexto: {context}\n\nPergunta: {question}"
)

### 5.3 Criando uma cadeia de recuperação

In [8]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

chain = (
    {"context": lambda x: retriever.get_relevant_documents(x["question"]), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


### 5.4 Executando a cadeia de recuperação

In [9]:
pergunta = "Quais são os principais desafios do RAG?"
resultado = chain.invoke({"question": pergunta})
print(resultado)

C:\Users\marie\AppData\Local\Temp\ipykernel_4696\1602426054.py:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  {"context": lambda x: retriever.get_relevant_documents(x["question"]), "question": RunnablePassthrough()}


O contexto fornecido não contém informações sobre os desafios do RAG (Retrieval Augmented Generation).  Portanto, não é possível responder à pergunta com base nas informações disponíveis.


## 6. Recuperação baseada em BM25

BM25 é uma função de classificação que mede relevância com base na frequência de termos e no tamanho do documento.

In [10]:
from langchain_community.retrievers import BM25Retriever

chunks = [
    "Python foi criado por Guido van Rossum em 1991.",
    "Python é uma linguagem popular para Machine Learning.",
    "A biblioteca PyTorch é amplamente usada em IA."
]

# criando um retriever baseado em BM25
bm25_retriever = BM25Retriever.from_texts(chunks, k=3)

# consultando o retriever
results = bm25_retriever.invoke("Quando o Python foi criado?")
print(results[0].page_content)


Python foi criado por Guido van Rossum em 1991.


## 7. Avaliação do RAG

A avaliação do RAG pode ser feita usando métricas como precisão de contexto e fidelidade da resposta

### 7.1 Fidelidade da resposta

In [11]:
from ragas.integrations.langchain import EvaluatorChain
from ragas.metrics import faithfulness
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

faithfulness_chain = EvaluatorChain(metric=faithfulness, llm=llm, embeddings=get_embedding_function())

eval_result = faithfulness_chain({
    "question": "Como o modelo RAG melhora a resposta a perguntas?",
    "answer": "O RAG melhora respostas combinando recuperação de documentos com LLMs.",
    "contexts": [
        "O modelo RAG integra recuperação de documentos para fornecer informações atualizadas.",
        "Ao incorporar mecanismos de busca, o RAG aproveita fontes externas de conhecimento."
    ]
})

print(f"Fidelidade: {eval_result['faithfulness']}")

c:\Users\marie\OneDrive\Área de Trabalho\projeto_libras\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\marie\AppData\Local\Temp\ipykernel_4696\2759611388.py:10: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  eval_result = faithfulness_chain({


Fidelidade: 0.0


In [12]:
from ragas.integrations.langchain import EvaluatorChain
from ragas.metrics import faithfulness, context_precision
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

context_precision_chain = EvaluatorChain(metric=context_precision, llm=llm, embeddings=get_embedding_function())

eval_result = context_precision_chain({
    "question": "Quais são os principais desafios do RAG?",
    "answer": "Os desafios do RAG incluem qualidade dos embeddings, recuperação eficaz e alucinações do modelo.",
    "contexts": [
        "Um desafio do RAG é garantir que os embeddings vetoriais representem bem a semântica do texto.",
        "Outro desafio é melhorar os algoritmos de recuperação para buscar os documentos mais relevantes.",
        "O RAG pode sofrer com alucinações caso os dados recuperados sejam insuficientes."
    ],
    "ground_truth": "Os principais desafios do RAG incluem a qualidade dos embeddings vetoriais, a eficácia da recuperação de documentos e a minimização de alucinações do modelo gerativo."
})


print(f"Precisão de Contexto: {eval_result['context_precision']:.2f}")

Precisão de Contexto: 0.00


## 8. RAG com grafos

Em vez de usar apenas vetores, podemos usar bancos de dados gráficos (como Neo4j) para estruturar e consultar o conhecimento armazenado.

### 8.1 Criando o banco de dados

In [13]:
from langchain_community.graphs import Neo4jGraph

load_dotenv('.env')
NEO4J_PASSWORD =os.environ.get("NEO4J_PASSWORD")

graph = Neo4jGraph(username="neo4j", password=NEO4J_PASSWORD)


C:\Users\marie\AppData\Local\Temp\ipykernel_4696\4219768223.py:6: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(username="neo4j", password=NEO4J_PASSWORD)


### 8.2 Armazenando documentos como grafos

In [14]:
from langchain_experimental.graph_transformers import LLMGraphTransformer


llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(documents)

graph.add_graph_documents(graph_documents, include_source=True, baseEntityLabel=True)

### 8.3 Consultando grafos com Cypher

In [15]:
results = graph.query("""
MATCH (`gemini-1.5-flash`:Model {id: "gemini-1.5-flash"})-[:DEVELOPED_BY]->(org:Organization)
RETURN org
""")

print(results)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: DEVELOPED_BY)} {position: line: 2, column: 61, offset: 61} for query: '\nMATCH (`gemini-1.5-flash`:Model {id: "gemini-1.5-flash"})-[:DEVELOPED_BY]->(org:Organization)\nRETURN org\n'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run thi

[]


## 9. Criando Graph RAG Chain

In [18]:
from langchain_community.chains.graph_qa.cypher import GraphCypherQAChain
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)


llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    convert_system_message_to_human=True,
    handle_parsing_errors=True,
    temperature=0.6,
    max_tokens= 1000,
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    },
)

chain = GraphCypherQAChain.from_llm(llm=llm, graph=graph, verbose=True, allow_dangerous_requests=True)

result = chain.invoke({"query": "O que é machine learning?"})
print(f"Resposta final: {result['result']}")




> Entering new GraphCypherQAChain chain...


c:\Users\marie\OneDrive\Área de Trabalho\projeto_libras\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:362: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Generated Cypher:
MATCH (d:Document) WHERE d.text CONTAINS "machine learning" RETURN d.text
Full Context:
[]


c:\Users\marie\OneDrive\Área de Trabalho\projeto_libras\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:362: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



> Finished chain.
Resposta final: Não sei a resposta.
